This script is for converting the matsim output files (generated by Aurore) into street graphs

In [15]:
import copy
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import geopandas as gpd
import shapely as shp
import pyproj
import numpy as np

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'SNMan','SNMan Shared','data_v2'
)
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
paper_path = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'E-Bike City Accessibility','EBC Accessibility Paper - Shared'
)

#matsim_results_path = os.path.join(
#    paper_path, 'MATSim results', '2024-07-12 Travel times before and after'
#)

matsim_results_path = os.path.join(
    paper_path, 'MATSim results', '2024-07-17 with link counts'
)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [16]:
TIME = '18:00'
STATE = 'before'

In [17]:
# read the original osm export
osm_export = snman.io.import_geofile_to_gdf(
    os.path.join(matsim_results_path, f'{STATE}_oneway_links_exploded.gzip')
)

In [18]:
# read the matsim output
tt = pd.read_csv(
    os.path.join(matsim_results_path, f'{STATE}_bike100pct.csv')
)
tt.set_index('OSM_ID', inplace=True)
tt

,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,N_cars_0:30,bike_0:30,...,N_bikes_22:30,car_23:00,N_cars_23:00,bike_23:00,N_bikes_23:00,car_23:30,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196
OSM_ID,,,,,,,,,,,,,,,,,,,,,
7749021,3640,15.39,25.64,NaN,0,NaN,0,NaN,0,NaN,...,0,22.0,14,NaN,0,22.0,12,NaN,0,NaN
7701086,3638,22.48,22.48,NaN,0,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
7701087,3639,11.96,11.96,NaN,0,NaN,0,NaN,0,NaN,...,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN
7701079,3630,40.03,40.03,NaN,0,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,0,46.0,1,NaN,0,NaN
7642172,3633,4.14,13.81,NaN,0,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7694151,36256,30.34,30.34,NaN,0,NaN,0,1.0,1,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
7822078,132450,19.23,19.23,NaN,0,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,0,25.5,2,NaN,0,NaN
7740686,3629,46.18,46.18,NaN,0,NaN,0,NaN,0,NaN,...,0,53.0,1,NaN,0,53.0,1,NaN,0,NaN


In [19]:
# join the matsim output with the input file
osm_export['osm_id'] = osm_export['osm_id'].astype('int64')

m = pd.merge(
    tt.reset_index(), osm_export[['osm_id', 'highway', 'geometry']],
    left_on='OSM_ID', right_on='osm_id', how='left'
)
m.drop(columns='osm_id', inplace=True)
m = gpd.GeoDataFrame(m)
m

,OSM_ID,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,N_cars_0:30,...,N_cars_23:00,bike_23:00,N_bikes_23:00,car_23:30,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry
0,7749021,3640,15.39,25.64,NaN,0,NaN,0,NaN,0,...,14,NaN,0,22.0,12,NaN,0,NaN,tertiary,"LINESTRING (2671251.848 1242229.884, 2671202.1..."
1,7701086,3638,22.48,22.48,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2683253.128 1261522.645, 2683332.6..."
2,7701087,3639,11.96,11.96,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2692815.897 1254306.119, 2692827.0..."
3,7701079,3630,40.03,40.03,NaN,0,NaN,0,NaN,0,...,0,NaN,0,46.0,1,NaN,0,NaN,residential,"LINESTRING (2684234.369 1260615.260, 2684241.2..."
4,7642172,3633,4.14,13.81,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,0,NaN,living_street,"LINESTRING (2711487.842 1241253.976, 2711536.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143697,7694151,36256,30.34,30.34,NaN,0,NaN,0,1.0,1,...,0,NaN,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2697083.477 1265914.727, 2697209.8..."
143698,7822078,132450,19.23,19.23,NaN,0,NaN,0,NaN,0,...,0,NaN,0,25.5,2,NaN,0,NaN,residential,"LINESTRING (2672724.778 1249865.551, 2672673.3..."
143699,7740686,3629,46.18,46.18,NaN,0,NaN,0,NaN,0,...,1,NaN,0,53.0,1,NaN,0,NaN,residential,"LINESTRING (2641479.497 1244490.798, 2641516.1..."
143700,7701078,3628,21.15,21.15,NaN,0,NaN,0,NaN,0,...,4,28.0,1,14.5,2,NaN,0,NaN,residential,"LINESTRING (2684233.087 1260764.503, 2684321.0..."


In [20]:
# create start and end points from line geometries
m['u'] = m.apply(
    lambda row: shp.Point(row['geometry'].coords[0]),
    axis=1
)
m['v'] = m.apply(
    lambda row: shp.Point(row['geometry'].coords[-1]),
    axis=1
)
m

,OSM_ID,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,N_cars_0:30,...,N_bikes_23:00,car_23:30,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry,u,v
0,7749021,3640,15.39,25.64,NaN,0,NaN,0,NaN,0,...,0,22.0,12,NaN,0,NaN,tertiary,"LINESTRING (2671251.848 1242229.884, 2671202.1...",POINT (2671251.848 1242229.884),POINT (2671163.959 1242189.480)
1,7701086,3638,22.48,22.48,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2683253.128 1261522.645, 2683332.6...",POINT (2683253.128 1261522.645),POINT (2683332.685 1261572.104)
2,7701087,3639,11.96,11.96,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2692815.897 1254306.119, 2692827.0...",POINT (2692815.897 1254306.119),POINT (2692827.003 1254257.555)
3,7701079,3630,40.03,40.03,NaN,0,NaN,0,NaN,0,...,0,46.0,1,NaN,0,NaN,residential,"LINESTRING (2684234.369 1260615.260, 2684241.2...",POINT (2684234.369 1260615.260),POINT (2684233.087 1260764.503)
4,7642172,3633,4.14,13.81,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,living_street,"LINESTRING (2711487.842 1241253.976, 2711536.8...",POINT (2711487.842 1241253.976),POINT (2711536.804 1241223.783)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143697,7694151,36256,30.34,30.34,NaN,0,NaN,0,1.0,1,...,0,NaN,0,NaN,0,NaN,residential,"LINESTRING (2697083.477 1265914.727, 2697209.8...",POINT (2697083.477 1265914.727),POINT (2697209.883 1265915.338)
143698,7822078,132450,19.23,19.23,NaN,0,NaN,0,NaN,0,...,0,25.5,2,NaN,0,NaN,residential,"LINESTRING (2672724.778 1249865.551, 2672673.3...",POINT (2672724.778 1249865.551),POINT (2672673.340 1249926.998)
143699,7740686,3629,46.18,46.18,NaN,0,NaN,0,NaN,0,...,0,53.0,1,NaN,0,NaN,residential,"LINESTRING (2641479.497 1244490.798, 2641516.1...",POINT (2641479.497 1244490.798),POINT (2641560.608 1244656.618)
143700,7701078,3628,21.15,21.15,NaN,0,NaN,0,NaN,0,...,1,14.5,2,NaN,0,NaN,residential,"LINESTRING (2684233.087 1260764.503, 2684321.0...",POINT (2684233.087 1260764.503),POINT (2684321.047 1260759.262)


In [21]:
# reconstruct nodes and give them new IDs
nd = pd.concat([m['u'], m['v']]).reset_index().rename(columns={0: 'geometry'})
nd['osmid'] = pd.factorize(nd['geometry'])[0]
nd[['x', 'y']] = nd.apply(
    lambda row: (row['geometry'].x, row['geometry'].y),
    axis=1,
    result_type='expand'
)
nd.drop(columns=['index'], inplace=True)
#nd.drop(columns='OSM_ID', inplace=True)
nd.drop_duplicates(inplace=True)
nd.set_index('osmid', inplace=True)
nd = gpd.GeoDataFrame(nd, geometry='geometry', crs=2056)

nd

,geometry,x,y
osmid,,,
0,POINT (2671251.848 1242229.884),2.671252e+06,1.242230e+06
1,POINT (2683253.128 1261522.645),2.683253e+06,1.261523e+06
2,POINT (2692815.897 1254306.119),2.692816e+06,1.254306e+06
3,POINT (2684234.369 1260615.260),2.684234e+06,1.260615e+06
4,POINT (2711487.842 1241253.976),2.711488e+06,1.241254e+06
...,...,...,...
56479,POINT (2678492.402 1253489.523),2.678492e+06,1.253490e+06
56480,POINT (2687567.606 1251017.449),2.687568e+06,1.251017e+06
56481,POINT (2687666.721 1250937.776),2.687667e+06,1.250938e+06


In [22]:
# write the new node IDs into the edge table and create an index like in the street graph

m2 = pd.merge(
    m.reset_index(), nd.reset_index()[['osmid', 'geometry']],
    how='left', left_on='u', right_on='geometry', suffixes=['', '_right']
).drop(columns=['u', 'geometry_right']).rename(columns={'osmid': 'u'})

m2 = pd.merge(
    m2, nd.reset_index()[['osmid', 'geometry']],
    how='left', left_on='v', right_on='geometry', suffixes=['', '_right']
).drop(columns=['v', 'geometry_right']).rename(columns={'osmid': 'v'})

m2['uv'] = m2.apply(lambda row: (row['u'], row['v']), axis=1)
m2['length'] = m2.apply(lambda row: row.geometry.length, axis=1)

m2

,index,OSM_ID,LinkId,FreeflowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,...,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry,u,v,uv,length
0,0,7749021,3640,15.39,25.64,NaN,0,NaN,0,NaN,...,12,NaN,0,NaN,tertiary,"LINESTRING (2671251.848 1242229.884, 2671202.1...",0,17108,"(0, 17108)",106.850626
1,1,7701086,3638,22.48,22.48,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,residential,"LINESTRING (2683253.128 1261522.645, 2683332.6...",1,12832,"(1, 12832)",93.677650
2,2,7701087,3639,11.96,11.96,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,residential,"LINESTRING (2692815.897 1254306.119, 2692827.0...",2,17850,"(2, 17850)",49.817573
3,3,7701079,3630,40.03,40.03,NaN,0,NaN,0,NaN,...,1,NaN,0,NaN,residential,"LINESTRING (2684234.369 1260615.260, 2684241.2...",3,56228,"(3, 56228)",166.786047
4,4,7642172,3633,4.14,13.81,NaN,0,NaN,0,NaN,...,0,NaN,0,NaN,living_street,"LINESTRING (2711487.842 1241253.976, 2711536.8...",4,14549,"(4, 14549)",57.522427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143697,143697,7694151,36256,30.34,30.34,NaN,0,NaN,0,1.0,...,0,NaN,0,NaN,residential,"LINESTRING (2697083.477 1265914.727, 2697209.8...",56236,28493,"(56236, 28493)",126.407084
143698,143698,7822078,132450,19.23,19.23,NaN,0,NaN,0,NaN,...,2,NaN,0,NaN,residential,"LINESTRING (2672724.778 1249865.551, 2672673.3...",56158,55951,"(56158, 55951)",80.134645
143699,143699,7740686,3629,46.18,46.18,NaN,0,NaN,0,NaN,...,1,NaN,0,NaN,residential,"LINESTRING (2641479.497 1244490.798, 2641516.1...",56235,23,"(56235, 23)",192.410583
143700,143700,7701078,3628,21.15,21.15,NaN,0,NaN,0,NaN,...,2,NaN,0,NaN,residential,"LINESTRING (2684233.087 1260764.503, 2684321.0...",56228,20679,"(56228, 20679)",88.115329


In [23]:
# replace "none" and "nan" travel times with free flow
# in case of paths, set car travel time to inf
import copy
m3 = copy.deepcopy(m2)

# fix a typo in the one column name
m3.rename(columns={'FreeflowTTCar': 'FreeFlowTTCar'}, inplace=True)

# build the 96 column names automatically
for mode in ['car', 'bike']:
    for hour in map(str, range(24)):
        for minute in ['00', '30']:
            column = f'{mode}_{hour}:{minute}'
            print(column)
            ff_tt_column = f'FreeFlowTT{mode.capitalize()}'
            # replace all nan values with the corresponding free flow travel time
            m3[column] = m3.apply(
                # cars on paths -> inf travel time
                lambda row: np.inf if row['highway'] == 'path' and mode == 'car'
                # nan -> free flow travel time
                else row[ff_tt_column] if np.isnan(row[column])
                # else -> no change
                else row[column],
                axis=1
            )
m3

car_0:00
car_0:30
car_1:00
car_1:30
car_2:00
car_2:30
car_3:00
car_3:30
car_4:00
car_4:30
car_5:00
car_5:30
car_6:00
car_6:30
car_7:00
car_7:30
car_8:00
car_8:30
car_9:00
car_9:30
car_10:00
car_10:30
car_11:00
car_11:30
car_12:00
car_12:30
car_13:00
car_13:30
car_14:00
car_14:30
car_15:00
car_15:30
car_16:00
car_16:30
car_17:00
car_17:30
car_18:00
car_18:30
car_19:00
car_19:30
car_20:00
car_20:30
car_21:00
car_21:30
car_22:00
car_22:30
car_23:00
car_23:30
bike_0:00
bike_0:30
bike_1:00
bike_1:30
bike_2:00
bike_2:30
bike_3:00
bike_3:30
bike_4:00
bike_4:30
bike_5:00
bike_5:30
bike_6:00
bike_6:30
bike_7:00
bike_7:30
bike_8:00
bike_8:30
bike_9:00
bike_9:30
bike_10:00
bike_10:30
bike_11:00
bike_11:30
bike_12:00
bike_12:30
bike_13:00
bike_13:30
bike_14:00
bike_14:30
bike_15:00
bike_15:30
bike_16:00
bike_16:30
bike_17:00
bike_17:30
bike_18:00
bike_18:30
bike_19:00
bike_19:30
bike_20:00
bike_20:30
bike_21:00
bike_21:30
bike_22:00
bike_22:30
bike_23:00
bike_23:30


,index,OSM_ID,LinkId,FreeFlowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,...,N_cars_23:30,bike_23:30,N_bikes_23:30,Unnamed: 196,highway,geometry,u,v,uv,length
0,0,7749021,3640,15.39,25.64,15.39,0,25.64,0,15.39,...,12,25.64,0,NaN,tertiary,"LINESTRING (2671251.848 1242229.884, 2671202.1...",0,17108,"(0, 17108)",106.850626
1,1,7701086,3638,22.48,22.48,22.48,0,22.48,0,22.48,...,0,22.48,0,NaN,residential,"LINESTRING (2683253.128 1261522.645, 2683332.6...",1,12832,"(1, 12832)",93.677650
2,2,7701087,3639,11.96,11.96,11.96,0,11.96,0,11.96,...,0,11.96,0,NaN,residential,"LINESTRING (2692815.897 1254306.119, 2692827.0...",2,17850,"(2, 17850)",49.817573
3,3,7701079,3630,40.03,40.03,40.03,0,40.03,0,40.03,...,1,40.03,0,NaN,residential,"LINESTRING (2684234.369 1260615.260, 2684241.2...",3,56228,"(3, 56228)",166.786047
4,4,7642172,3633,4.14,13.81,4.14,0,13.81,0,4.14,...,0,13.81,0,NaN,living_street,"LINESTRING (2711487.842 1241253.976, 2711536.8...",4,14549,"(4, 14549)",57.522427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143697,143697,7694151,36256,30.34,30.34,30.34,0,30.34,0,1.00,...,0,30.34,0,NaN,residential,"LINESTRING (2697083.477 1265914.727, 2697209.8...",56236,28493,"(56236, 28493)",126.407084
143698,143698,7822078,132450,19.23,19.23,19.23,0,19.23,0,19.23,...,2,19.23,0,NaN,residential,"LINESTRING (2672724.778 1249865.551, 2672673.3...",56158,55951,"(56158, 55951)",80.134645
143699,143699,7740686,3629,46.18,46.18,46.18,0,46.18,0,46.18,...,1,46.18,0,NaN,residential,"LINESTRING (2641479.497 1244490.798, 2641516.1...",56235,23,"(56235, 23)",192.410583
143700,143700,7701078,3628,21.15,21.15,21.15,0,21.15,0,21.15,...,2,21.15,0,NaN,residential,"LINESTRING (2684233.087 1260764.503, 2684321.0...",56228,20679,"(56228, 20679)",88.115329


In [24]:
# write the chosen travel time window as master travel time
m3[f'cost_lanes_{MODE_PRIVATE_CARS}_{DIRECTION_FORWARD}'] = m3[f'car_{TIME}']
m3[f'cost_lanes_{MODE_PRIVATE_CARS}_{DIRECTION_BACKWARD}'] = np.inf

# set some attributes
m3['oneway'] = 'yes'

# add a simple single motorized lane in the forward direction,
# please note that this street graph is only needed to represent car travel times
# so the exact number of lanes does not matter
m3['lanes'] = m3.apply(
    lambda row: snman.space_allocation.SpaceAllocation(
        [snman.space_allocation.Lane(LANETYPE_MOTORIZED, DIRECTION_FORWARD)]
    ) if row['highway'] != 'path'
    else snman.space_allocation.SpaceAllocation(
        [snman.space_allocation.Lane(LANETYPE_FOOT, DIRECTION_FORWARD)]
    ),
    axis=1
)

m3.set_index('uv', inplace=True)
m3

,index,OSM_ID,LinkId,FreeFlowTTCar,FreeFlowTTBike,car_0:00,N_cars_0:00,bike_0:00,N_bikes_0:00,car_0:30,...,Unnamed: 196,highway,geometry,u,v,length,cost_lanes_private_cars_>,cost_lanes_private_cars_<,oneway,lanes
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 17108)",0,7749021,3640,15.39,25.64,15.39,0,25.64,0,15.39,...,NaN,tertiary,"LINESTRING (2671251.848 1242229.884, 2671202.1...",0,17108,106.850626,22.00,inf,yes,[M>*3.0]
"(1, 12832)",1,7701086,3638,22.48,22.48,22.48,0,22.48,0,22.48,...,NaN,residential,"LINESTRING (2683253.128 1261522.645, 2683332.6...",1,12832,93.677650,22.48,inf,yes,[M>*3.0]
"(2, 17850)",2,7701087,3639,11.96,11.96,11.96,0,11.96,0,11.96,...,NaN,residential,"LINESTRING (2692815.897 1254306.119, 2692827.0...",2,17850,49.817573,11.96,inf,yes,[M>*3.0]
"(3, 56228)",3,7701079,3630,40.03,40.03,40.03,0,40.03,0,40.03,...,NaN,residential,"LINESTRING (2684234.369 1260615.260, 2684241.2...",3,56228,166.786047,23.50,inf,yes,[M>*3.0]
"(4, 14549)",4,7642172,3633,4.14,13.81,4.14,0,13.81,0,4.14,...,NaN,living_street,"LINESTRING (2711487.842 1241253.976, 2711536.8...",4,14549,57.522427,4.14,inf,yes,[M>*3.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(56236, 28493)",143697,7694151,36256,30.34,30.34,30.34,0,30.34,0,1.00,...,NaN,residential,"LINESTRING (2697083.477 1265914.727, 2697209.8...",56236,28493,126.407084,1.00,inf,yes,[M>*3.0]
"(56158, 55951)",143698,7822078,132450,19.23,19.23,19.23,0,19.23,0,19.23,...,NaN,residential,"LINESTRING (2672724.778 1249865.551, 2672673.3...",56158,55951,80.134645,26.00,inf,yes,[M>*3.0]
"(56235, 23)",143699,7740686,3629,46.18,46.18,46.18,0,46.18,0,46.18,...,NaN,residential,"LINESTRING (2641479.497 1244490.798, 2641516.1...",56235,23,192.410583,53.00,inf,yes,[M>*3.0]


In [25]:
m3 = m3[[
    'OSM_ID', 'FreeFlowTTCar', 'car_7:00', 'car_18:00', 'N_cars_7:00', 'N_cars_18:00', 'highway', 'length', 'lanes', 'oneway', 'geometry'
]]

In [26]:
# create a new street graph
G = snman.street_graph.street_graph_from_gdf(nd, m3)

In [27]:
# save the street graph

if 1:
    snman.io.export_street_graph(
        G,
        os.path.join(outputs_path, f'tt_{STATE}_edges.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gpkg'),
        crs=CRS_for_export,
        stringify_additional_attributes=['lanes']
    )

In [28]:
STATE

'before'